In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix,accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

df = pd.read_csv('./train1.csv')
X_train = df.drop(['target','id'], axis=1)


In [2]:
y_train = df['target']

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=29173, stratify=y_train)

In [4]:
X_test.shape

(88863, 65)

In [5]:
"""df_test = pd.read_csv('test.csv')
X_test = df_test.drop('id',axis=1) 
test_ids = df_test['id']"""

"df_test = pd.read_csv('test.csv')\nX_test = df_test.drop('id',axis=1) \ntest_ids = df_test['id']"

In [6]:
categorical_cols = [col for col in X_train.columns if col.endswith('_cat')]
X_train[categorical_cols] = X_train[categorical_cols].astype('category')
X_test[categorical_cols] = X_test[categorical_cols].astype('category')
binary_cols = [col for col in X_train.columns if col.endswith('_bin')]
X_train[binary_cols] = X_train[binary_cols].astype(bool)
X_test[binary_cols] = X_test[binary_cols].astype(bool)

In [7]:
def simple_imputer(X_train, X_val, categorical_columns=None, numerical_columns=None):
    """
    Impute missing values in train and validation/test sets using only training data statistics.
    - categorical_columns: list of categorical columns to impute with mode
    - numerical_columns: list of numerical columns to impute with mean
    """
    X_train_imputed = X_train.copy()
    X_val_imputed = X_val.copy()

    # Categorical columns
    if categorical_columns:
        for col in categorical_columns:
            if col in X_train_imputed.columns:
                mode_value = X_train_imputed[col].mode(dropna=True)
                if not mode_value.empty:
                    mode_value = mode_value[0]
                    X_train_imputed[col].fillna(mode_value, inplace=True)
                    if col in X_val_imputed.columns:
                        X_val_imputed[col].fillna(mode_value, inplace=True)

    # Numerical columns
    if numerical_columns:
        for col in numerical_columns:
            if col in X_train_imputed.columns:
                mean_value = X_train_imputed[col].mean(skipna=True)
                if pd.notna(mean_value):  # ensure mean is valid
                    X_train_imputed[col].fillna(mean_value, inplace=True)
                    if col in X_val_imputed.columns:
                        X_val_imputed[col].fillna(mean_value, inplace=True)

    return X_train_imputed, X_val_imputed


In [8]:
num_cols = [col for col in X_train.columns if not col.endswith(('_cat', '_bin'))]

In [9]:
X_train_imputed,X_test_imputed = simple_imputer(X_train, X_test, categorical_cols,num_cols)

In [10]:
X_train_imputed['ps_car_12'].isnull().sum()

np.int64(0)

In [59]:
X_test_imputed.isna().sum()[X_test.isna().sum() > 0]


ps_ind_02_cat    0
ps_ind_04_cat    0
ps_ind_05_cat    0
ps_car_01_cat    0
ps_car_03_cat    0
ps_car_05_cat    0
ps_car_07_cat    0
ps_car_09_cat    0
ps_reg_03        0
ps_car_11        0
ps_car_12        0
ps_car_14        0
feature4         0
dtype: int64

In [11]:
baseline_model = RandomForestClassifier(
    n_estimators=100,        
    max_depth=None,        
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',     
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

baseline_model.fit(X_train_imputed, y_train)
y_pred_proba = baseline_model.predict_proba(X_test_imputed)[:, 1]  # probabilities for positive class

roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Baseline AUROC:", roc_auc)

Baseline AUROC: 0.5879613777869592


In [12]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 20, 30],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt'],
    'bootstrap': [True],
    'class_weight': [None]
}


In [13]:

rf = RandomForestClassifier(random_state=29173)

Random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=36,             
    scoring='roc_auc',       
    cv=3,
    verbose=2,
    random_state=29173,
    n_jobs=-1
)


Random_search.fit(X_train_imputed, y_train)

print("Best parameters:", Random_search.best_params_)
print("Best roc_auc score:", Random_search.best_score_)


d:\ML project\.venv\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 36 is smaller than n_iter=50. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'class_weight': None, 'bootstrap': True}
Best roc_auc score: 0.6131577192632487
